<a href="https://colab.research.google.com/github/NatsuiroGinga/pycorrector/blob/master/pycorrector_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 文本纠错

## 1 下载与导入

In [ ]:
!pip install pandas transformers datasets peft accelerate pycorrector -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 27.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install gradio

In [ ]:
import sys
import gradio as gr
import torch
from transformers import BertTokenizerFast, BertForMaskedLM
from pycorrector import MacBertCorrector
from pycorrector.gpt.gpt_corrector import GptCorrector

## 2 Gradio

In [ ]:
def predict(text):
    return model.correct(text)


if __name__ == '__main__':
    model = MacBertCorrector()

    examples = [
        ['真麻烦你了。希望你们好好的跳无'],
        ['少先队员因该为老人让坐'],
        ['机七学习是人工智能领遇最能体现智能的一个分知'],
        ['今天心情很好'],
        ['他法语说的很好，的语也不错'],
        ['他们的吵翻很不错，再说他们做的咖喱鸡也好吃'],
    ]

    gr.Interface(
        predict,
        inputs="text",
        outputs="text",
        title="Chinese Spelling Correction Model",
        description="Copy or input error Chinese text. Submit and the machine will correct text.",
        article="Link to github: <a href='https://github.com/shibing624/pycorrector' style='color:blue;' target='_blank\'>pycorrector</a>",
        examples=examples
    ).launch()

NameError: name 'MacBertCorrector' is not defined

## 3 MacBert-CSC

In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizerFast.from_pretrained("shibing624/macbert4csc-base-chinese")
model = BertForMaskedLM.from_pretrained("shibing624/macbert4csc-base-chinese")
model.to(device)

texts = ["C请介绍一下", "你找到你最喜欢的工作，我也很高心。"]
with torch.no_grad():
    outputs = model(**tokenizer(texts, padding=True, return_tensors='pt').to(device))

result = []
for ids, text in zip(outputs.logits, texts):
    _text = tokenizer.decode(torch.argmax(ids, dim=-1), skip_special_tokens=True).replace(' ', '')
    corrected_text = _text[:len(text)]
    print(text, ' => ', corrected_text)
    result.append(corrected_text)
print(result)

C请介绍一下  =>  c请介绍一下
你找到你最喜欢的工作，我也很高心。  =>  你找到你最喜欢的工作，我也很高兴。
['c请介绍一下', '你找到你最喜欢的工作，我也很高兴。']


## 4 GPT/Qwen

In [ ]:
prompt = """
# 系统角色指令： 你是一名专业的中文文本纠错模型，能够识别并纠正文本中的错别字、多字、漏字，以及常见词汇和领域术语的误用。

# 对模型的主要要求：

1. 请专注于纠正输入文本中的语言错误，尽量保持原意。

2. 当出现领域专有名词（如公司名称、产品名称、技术术语等）时，请参照以下上下文信息进行纠正。

3. 对于没有明显错误或不影响理解的部分，不要进行改动。

4. 仅输出纠正后的文本，不要输出任何额外解释或标记。

# 上下文信息（示例）：

1. 真视通：公司名，与“真通智用”“整通只用” 等易混淆

2. 紫荆视通：公司名，与“紫金系统”等相近但不相同

3. 双相液冷板：正确技术名词，与“双像叶冷板”“双向叶冷板” 相混淆

"""

print(prompt)


# 系统角色指令： 你是一名专业的中文文本纠错模型，能够识别并纠正文本中的错别字、多字、漏字，以及常见词汇和领域术语的误用。

# 对模型的主要要求：

1. 请专注于纠正输入文本中的语言错误，尽量保持原意。

2. 当出现领域专有名词（如公司名称、产品名称、技术术语等）时，请参照以下上下文信息进行纠正。

3. 对于没有明显错误或不影响理解的部分，不要进行改动。

4. 仅输出纠正后的文本，不要输出任何额外解释或标记。

# 上下文信息（示例）：

1. 真视通：公司名，与“真通智用”“整通只用” 等易混淆

2. 紫荆视通：公司名，与“紫金系统”等相近但不相同

3. 双相液冷板：正确技术名词，与“双像叶冷板”“双向叶冷板” 相混淆




In [ ]:
m = GptCorrector()
print(m.correct_batch(sentences=['请下真是通的主要业务'], prefix_prompt=prompt))

NameError: name 'GptCorrector' is not defined

# 数据增强

## 1 下载与导入

In [4]:
!pip install jionlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 MB 6.7 MB/s eta 0:00:00


In [5]:
!git clone https://huggingface.co/datasets/Heehobino/transtrue_text_correction

Cloning into 'transtrue_text_correction'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 22 (delta 9), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (22/22), 14.51 KiB | 1.81 MiB/s, done.


In [6]:
import jionlp as jio
from concurrent.futures import ThreadPoolExecutor
from huggingface_hub import HfApi

# jionlp - 微信公众号: JioNLP  Github: `https://github.com/dongrixinyu/JioNLP`.
# jiojio - `http://www.jionlp.com/jionlp_online/cws_pos` is available for online trial.
# jiojio - Successfully load C funcs for CWS and POS acceleration.


## 2 同音字替换

In [74]:
homophone_substitution_config = {
    "input_file_path": "/content/transtrue_text_correction/ground_truth.txt",
    "output_file_path": "/content/transtrue_text_correction/train.txt",
    "augmentation_num": 20,
    "homo_ratio": 0.5,
    "allow_mispronounce": True,
}

print(jio.homophone_substitution.__doc__)


    采用同音词汇进行原文替换，达到数据增强的目的。

    原理简述：汉语输入法中，拼音输入法为目前使用最广泛的一种打字法，使用率占比约 97%。
        在实际使用中，常常出现同音词的打字错误，例如：原句为
        “人口危机如果无法得到及时解决，80后、90后们将受到巨大的冲击”，拼音输入法结果为
        “人口危机如果无法得到即时解决，80后、90后门将受到巨大的冲击”。
        从输入的错误来看，完全不影响人的阅读理解。
        因此，可以利用同音词汇替换，达到数据增强的目的。

        该工具中，方法具体实施时：
        1、不考虑拼音声调，因为拼音输入法基本不输入声调；
        2、考虑常见方言读音误读，如 zh 与 z 不分，eng 与 en 不分，f 与 h 不分，l 与 n 不分等情况；
        3、替换时，优先使用常用词汇（依据词频而定）；原因在于拼音输入法优先以常见词进行替换。

    Args:
        text(str): 原始文本
        augmentation_num(int): 数据增强对该条样本的扩展个数，默认为 3
        homo_ratio(float): 对每一个词汇的同音词替换概率，默认为 0.02
        allow_mispronounce(bool): 是否允许方言读音误读，如 zh 与 z 卷舌不分，默认为 True，允许词汇错音
        seed(int): 控制随机替换词汇每次不变，默认为 1，当为 0 时，每次调用产生结果不固定

    Returns:
        list(str): 数据增强的结果，特殊情况可以为空列表

    Examples:
        >>> import jionlp as jio
        >>> res = jio.homophone_substitution(
                      '中国驻英记者一向恪守新闻职业道德，为增进中英两国人民之间的了解和沟通发挥了积极作用。')
        >>> print(res)

        # ['中国驻英记者一向刻手信问职业道德，为增进中英两国人民之间的了解和沟通

### 2.1 测试

In [89]:
# 原始真实数据
ground_truth = "请介绍下真通智用公司"
# 生成同音词替换后的数据
res = jio.homophone_substitution(ground_truth, augmentation_num=5, homo_ratio=0.5)
print(res)

['请介绍吓真通智用公示', '请介绍侠政通支用公司', '请介绍下阵痛智用公司', '请介绍下镇痛只用公示', '轻介绍下真通只用公司']


### 2.2 并行生成数据

In [75]:
def process_line(line):
    return jio.homophone_substitution(line, augmentation_num=homophone_substitution_config["augmentation_num"], homo_ratio=homophone_substitution_config["homo_ratio"])

with open(homophone_substitution_config["input_file_path"], "r", encoding="utf-8") as infile, open(homophone_substitution_config["output_file_path"], "w", encoding="utf-8") as outfile:
  with ThreadPoolExecutor() as executor:
        results = executor.map(process_line, infile)  # 并发处理每行数据
        for r in results:
          outfile.writelines(r)  # 一次性写入文件

## 3 随机增删字符

In [97]:
print(jio.random_add_delete.__doc__)

 随机增删字符。
    随机在文本中增加、删除某个字符。

    原理简述：在文本中随机增加、删除某些不影响原意的字符，对文本语义不造成影响。
        例如：“23日，山东省监狱管理局原副局长王文杰等5人玩忽职守”，增删为
             "2日，山东监狱 管理局、原副局长文杰等5人玩忽职守.."。
        随机增加的字符的选择，依据对海量文本统计字符分布规律的 char_distribution.json
        文件得到，取其中的非中文字符进行添加；该分布经过了修饰，修饰方法参考
        self._prepare 方法内的注释。

    注意事项：
        1、对于某些 NLP 任务，如抽取其中时间词汇，则以上方法很容易干扰关键时间信息，
          故方法失效。待后续优化，引入控制参数，避免某类关键信息（时间、地点等被增删）。
        2、除了增删外，有一种同义词替换，本工具未采用，原因在于对语言的通畅性与语义影响
          过大，几乎找不到可用的增强文本。
          例如：“这个东西是干什么用的？”，根据同义词词林，“东西”的同义词包括“家伙”、“货色”、
               “小崽子”、“杂种”等，“这个“ 的同义词包括”此“、”斯“等。随机替换后的结果会出现
               非常离谱的文本，如 ”斯小崽子是干什么用的？“。
          经统计，同义词替换方法的语法不连贯与语义不明确比例占总数据量的 85%，
          因此本工具不采用同义词替换方法。

    Args:
        augmentation_num(int): 数据增强对该条样本的扩展个数，默认为 3
        seed(int): 控制随机交换位置每次不变，默认为 1，当为 0 时，每次调用产生结果不固定
        add_ratio(float): 对每一个位置随机增加字符概率，默认为 0.02
        delete_ratio(float): 对每一个汉字随机做删除的概率，默认为 0.02

    Returns:
        list(str): 数据增强的结果，特殊情况可以为空列表

    Examples:


In [11]:
random_add_delete_config = {
    "input_file_path": "/content/transtrue_text_correction/ground_truth.txt",
    "output_file_path": "/content/transtrue_text_correction/train.txt",
    "augmentation_num": 10,
    "add_ratio": 0.1,
    "delete_ratio": 0.1,
}

In [12]:
res = jio.random_add_delete('请介绍下真通智用公司', augmentation_num=random_add_delete_config["augmentation_num"],
add_ratio=random_add_delete_config["add_ratio"], delete_ratio=random_add_delete_config["delete_ratio"])

print(res)

['请D介3绍下真通ใ智用公司', '请2绍a下真通用公司', '请介下真y通智用公s司', '请介绍下真智用公']


## 4 回译数据增强

In [23]:
print(baidu_api.__doc__)  # 查看接口说明

baidu_api = jio.BaiduApi(
        [{'appid': '20250217002276622',
          'secretKey': 'dpCCwxOhV3uqFR76aLp7'}], gap_time=0.5)

apis = [baidu_api]
back_trans = jio.BackTranslation(mt_apis=apis)

 百度翻译 api 的调用接口

    参考文档：https://api.fanyi.baidu.com/doc/21
    支持语言：中文(zh)、英文(en)、西班牙语(spa)、德文(de)、法语(fra)、
            日语(jp)、俄语(ru)、葡萄牙语(pt)
    Args:
        from_lang: 输入源语言
        to_lang: 输入目标语言

    Return:

    Examples:
        >>> baidu_api = BaiduApi(
                [{'appid': '20200618000498778',
                 'secretKey': 'raHalLakgYitNuzGOoBZ'}])
        >>> text = '她很好看。'
        >>> res = baidu_api(text, from_lang='zh', to_lang='en')
        >>> print(res)

        # She looks good.

    


In [ ]:
text = '请介绍下真视通'
result = back_trans(text)

print(result)